# JULIA MPI First Example: pi computaton

My first example was to reproduce <a href="http://www.mcs.anl.gov/research/projects/mpi/tutorial/mpiexmpl/src/pi/C/solution.html">
the classic mypi </a> in the notebook

In [7]:
using MPIClusterManagers,MPI, Distributed

In [8]:
manager=MPIWorkerManager(10)
addprocs(manager)

10-element Vector{Int64}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [11]:
nprocs()
@mpi_do manager using MPI

In [12]:
@mpi_do manager comm = MPI.COMM_WORLD
#
# Enter number of intervals, and tell every processor
# Traditional MPI would do this with a BCAST
 @mpi_do manager n = rand()

In [14]:
# Let's see if the processors got it
@mpi_do manager println(n)

      From worker 7:	0.7406074814696212
      From worker 4:	0.040827179178438366
      From worker 10:	0.5515707242437361
      From worker 11:	0.9938286487567941
      From worker 3:	0.08258105202520172
      From worker 6:	0.25095230155615667
      From worker 5:	0.6302917747226593
      From worker 2:	0.29662265656751996
      From worker 9:	0.6856460982146318
      From worker 8:	0.6553394665867878


In [15]:
# my MPI id
@mpi_do manager myid = MPI.Comm_rank(comm)
@mpi_do manager println(myid)

      From worker 7:	5
      From worker 4:	2
      From worker 6:	4
      From worker 11:	9
      From worker 9:	7
      From worker 10:	8
      From worker 8:	6
      From worker 2:	0
      From worker 5:	3
      From worker 3:	1


In [16]:
# Get the number of processors
@mpi_do manager np = MPI.Comm_size(comm)
@mpi_do manager println(np)

      From worker 4:	10
      From worker 8:	10
      From worker 5:	10
      From worker 7:	10
      From worker 2:	10
      From worker 9:	10
      From worker 6:	10
      From worker 11:	10
      From worker 10:	10
      From worker 3:	10


Compute $\int_0^1 4/(1+x^2) dx= 4 atan(x)]_0^1$ which evaluates to $\pi$

In [10]:
using Interact

┌ Info: Precompiling Interact [c601a237-2ae4-5e1e-952c-7a85b0c7eef1]
└ @ Base loading.jl:1189


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/00368592c11ecb190ee1c0ddf27ce3f1255469c1-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/00368592c11ecb190ee1c0ddf27ce3f1255469c1-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [22]:
@mpi_do manager @show mypi

      From worker 4:	mypi = 0.3141592753589915
      From worker 2:	mypi = 0.31415928335899374
      From worker 5:	mypi = 0.3141592713589613
      From worker 11:	mypi = 0.31415924735898243
      From worker 7:	mypi = 0.31415926335898003
      From worker 10:	mypi = 0.31415925135898265
      From worker 6:	mypi = 0.31415926735896743
      From worker 8:	mypi = 0.3141592593589629
      From worker 9:	mypi = 0.3141592553590055
      From worker 3:	mypi = 0.31415927935894233


In [49]:
@time @mpi_do manager mypi, our_π = let
    n = 50_000_000
    comm = MPI.COMM_WORLD
    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : n 
        x = (i - .5)/n 
        s += 4/(1 + x^2) 
    end
    mypi = s/n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    if myid==0
        println(our_π - π) 
    end
    mypi, our_π
end

      From worker 2:	-2.3092638912203256e-14
  0.053561 seconds (1.73 k allocations: 146.625 KiB)


In [50]:
@mpi_do manager @show our_π

      From worker 4:	our_π = nothing
      From worker 5:	our_π = nothing
      From worker 7:	our_π = nothing
      From worker 6:	our_π = nothing
      From worker 2:	our_π = 3.14159265358977
      From worker 8:	our_π = nothing
      From worker 9:	our_π = nothing
      From worker 3:	our_π = nothing
      From worker 10:	our_π = nothing
      From worker 11:	our_π = nothing


In [54]:
our_π =  @fetchfrom 2 our_π

3.14159265358977

In [55]:
our_π

3.14159265358977

In [57]:
function f_serial()
    n = 50_000_000
    h = 1/n
    our_π = 0.0
    for i = 0:h:1
        our_π += 4/(1 + i^2)
    end
    our_π*h
end

function f_serial2(n)
    our_π = 0.0
    for i = 1:n
        x = (i - 0.5)/n
        our_π += 4/(1 + x^2)
    end
    our_π/n
end

f_serial2 (generic function with 1 method)

In [59]:
f_serial() #warmup
f_serial()
f_serial2(50_000_000) #warmup
@time f_serial2(50_000_000)

  0.049509 seconds


3.1415926535895617

In [61]:
function f_par(n)

 @mpi_do manager begin
    comm = MPI.COMM_WORLD
       
    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : $n 
        x = (i - .5)/$n 
        global s += 4/(1 + x^2) 
    end
    mypi = s/$n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    #if myid==0
     #   println(our_π - π) 
   # end
end
@fetchfrom 2 our_π   
end

f_par (generic function with 1 method)

In [65]:
@mpi_do manager function _pi_sum_par(n)
    comm = MPI.COMM_WORLD

    s = 0.0
    for i = MPI.Comm_rank(comm) + 1 : MPI.Comm_size(comm) : n
        x = (i - .5)/n 
        s += 4/(1 + x^2) 
    end
    mypi = s/n
    our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
    return our_π
end
function f_par2(n)
    @mpi_do manager tmp = _pi_sum_par($n)
    @fetchfrom 2 tmp
end
f_par(50_000_000) #warmup
f_par(50_000_000)
f_par2(50_000_000) #warmup
@time f_par2(50_000_000)

LoadError: TaskFailedException

[91m    nested task error: [39mOn worker 5:
    UndefVarError: `n` not defined in `MPIClusterManagers`
    Suggestion: check for spelling errors or missing imports.
    Stacktrace:
     [1] [0m[1m#99[22m
    [90m   @[39m [90m~/.julia/packages/MPIClusterManagers/RQVkV/src/[39m[90m[4mmpido.jl:40[24m[39m
     [2] [0m[1m#invokelatest#2[22m
    [90m   @[39m [90m./[39m[90m[4messentials.jl:1055[24m[39m[90m [inlined][39m
     [3] [0m[1minvokelatest[22m
    [90m   @[39m [90m./[39m[90m[4messentials.jl:1052[24m[39m
     [4] [0m[1m#107[22m
    [90m   @[39m [90m~/.julia/juliaup/julia-1.11.1+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Distributed/src/[39m[90m[4mprocess_messages.jl:283[24m[39m
     [5] [0m[1mrun_work_thunk[22m
    [90m   @[39m [90m~/.julia/juliaup/julia-1.11.1+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Distributed/src/[39m[90m[4mprocess_messages.jl:70[24m[39m
     [6] [0m[1mrun_w

In [16]:
π

π = 3.1415926535897...

In [66]:
[f_par2(10^k) for k=3:9] .- π

LoadError: TaskFailedException

[91m    nested task error: [39mOn worker 5:
    UndefVarError: `n` not defined in `MPIClusterManagers`
    Suggestion: check for spelling errors or missing imports.
    Stacktrace:
     [1] [0m[1m#115[22m
    [90m   @[39m [90m~/.julia/packages/MPIClusterManagers/RQVkV/src/[39m[90m[4mmpido.jl:40[24m[39m
     [2] [0m[1m#invokelatest#2[22m
    [90m   @[39m [90m./[39m[90m[4messentials.jl:1055[24m[39m[90m [inlined][39m
     [3] [0m[1minvokelatest[22m
    [90m   @[39m [90m./[39m[90m[4messentials.jl:1052[24m[39m
     [4] [0m[1m#107[22m
    [90m   @[39m [90m~/.julia/juliaup/julia-1.11.1+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Distributed/src/[39m[90m[4mprocess_messages.jl:283[24m[39m
     [5] [0m[1mrun_work_thunk[22m
    [90m   @[39m [90m~/.julia/juliaup/julia-1.11.1+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Distributed/src/[39m[90m[4mprocess_messages.jl:70[24m[39m
     [6] [0m[1mrun_

In [67]:
using DistributedArrays

LoadError: ArgumentError: Package DistributedArrays not found in current path.
- Run `import Pkg; Pkg.add("DistributedArrays")` to install the DistributedArrays package.

In [67]:
using Elemental
using LinearAlgebra

In [65]:
A = drandn(1000, 800);

In [68]:
svdvals(A)

800×1 DArray{Float64,2,Array{Float64,2}}:
 59.7715702168605   
 59.496075555258884 
 59.04585265227774  
 58.87963945925137  
 58.602996778644055 
 58.24142861506018  
 57.95876505405129  
 57.72555531429163  
 57.6616153792119   
 57.50109958680264  
 57.30636730358922  
 57.234205186637254 
 57.05905338286139  
  ⋮                 
  4.52099185543084  
  4.497618544387572 
  4.311303886910574 
  4.287312503144409 
  4.231855977271004 
  4.0521396338209765
  3.925727382976595 
  3.8515802245832838
  3.8152052520442   
  3.646071383932354 
  3.5329358075557353
  3.4800501933452885

In [ ]:
x